# Estimate PKAc levels

This code runs the Dopamine D1 cascade for the low (4Hz) and high (20Hz) optogenetic stimulation of DA neurons. The purpose is to see what levels of PKAc we get, and the time dynamics.

In [1]:
import os
from snudda import Snudda

neuron = "dspn"
# neuron = "ispn"

neuron_path = os.path.join("..", "..", "snudda_data", "neurons", "striatum", neuron)
network_path = os.path.join("networks", "check_PKAc")

In [ ]:
snudda = Snudda(network_path=network_path)
si = snudda.init_tiny(neuron_paths=neuron_path, neuron_names=neuron, number_of_neurons=[5], 
                      random_seed=1234)

si.network_data["regions"]["Cube"]["neurons"]["dspn"]["reaction_diffusion"] = "../../data/JSON/reaction_diffusion_D1_from_SBTab.json"
# si.network_data["regions"]["Cube"]["neurons"]["ispn"]["reaction_diffusion"] = "data/JSON/reaction_diffusion_D2.json-updated"


# How the ion channels are modified by DA
# OBS, we include SK direkt modulation, in relality it should be modulated by DA acting on Ca 
# si.network_data["regions"]["Cube"]["neurons"][neuron]["modulation"] = "modulation_parameters.json"
si.network_data["regions"]["Cube"]["neurons"][neuron]["modulation"] = "../../data/JSON/modulation_parameters-v2.json"

si.network_data["regions"]["Cube"]["neurons"][neuron]["modulation_key"] = "abc"

si.write_json()

snudda.create_network()

In [2]:
# Free memory
snudda = None

mech_dir = "/home/hjorth/HBP/BasalGangliaData/data/neurons/mechanisms"
sample_dt = None # 0.00005

sim_config_low = os.path.join("..", "..", "data", "JSON", "estimate_PKAc_4Hz.json")
sim_config_high = os.path.join("..", "..", "data", "JSON", "estimate_PKAc_20Hz.json")

sim_output_neuromodulation_low = os.path.join(network_path, "simulation", "output_neuromodulation_PKAc_4Hz.hdf5")
sim_output_neuromodulation_high = os.path.join(network_path, "simulation", "output_neuromodulation_PKAc_20Hz.hdf5")

sim_time = 10
n_workers = 5

In [ ]:
!nrnivmodl ../../snudda_data/neurons/mechanisms/

In [ ]:
run_str_low = f"mpirun -n {n_workers} snudda simulate {network_path} --time {sim_time} --simulation_config {sim_config_low} --mechdir {mech_dir} --enable_rxd_neuromodulation"
print(run_str_low)

In [ ]:
os.system(run_str_low)

In [ ]:
run_str_high = f"mpirun -n {n_workers} snudda simulate {network_path} --time {sim_time} --simulation_config {sim_config_high} --mechdir {mech_dir} --enable_rxd_neuromodulation"
print(run_str_high)

In [ ]:
os.system(run_str_high)

In [3]:
from snudda.utils import SnuddaLoadSimulation

nd = SnuddaLoadSimulation(sim_output_neuromodulation_low)
time = nd.get_time()
data_pka = nd.get_data("PKAc", 0)[0][0]
data_da = nd.get_data("DA", 0)[0][0]
data_da_external = nd.get_data("DA", 0)[0][0]

# This is saved with add_rxd_internal_concentration_recording_all -- check that it worked 
data_pka_all0 = nd.get_data("PKAc", 0)[0][0]

Loading networks/check_PKAc/simulation/output_neuromodulation_PKAc_4Hz.hdf5
WARNING. Depolarisation block in neuron - neuron_id: (name, parameter_key, morphology_key):
0: (dspn_0, pcfabf7dd, m4a48752c)
1: (dspn_1, pa925bef7, mfd0a1315)
2: (dspn_2, pb7178977, m93f282f3)
3: (dspn_3, p31fdf560, mfa814971)
4: (dspn_3, pd45a06eb, me61ede90)


In [4]:
nd.list_data_types(0)

['AC5',
 'AC5_ATP',
 'AC5_Ca',
 'AC5_Ca_ATP',
 'AC5_Ca_GaolfGTP',
 'AC5_Ca_GaolfGTP_ATP',
 'AC5_GaolfGTP',
 'AC5_GaolfGTP_ATP',
 'AMP',
 'ARPP21',
 'ATP',
 'B56PP2A',
 'B56PP2A_D32p75',
 'B56PP2A_pARPP21',
 'B56PP2Ap',
 'B56PP2Ap_D32p75',
 'B56PP2Ap_pARPP21',
 'B72PP2A',
 'B72PP2A_Ca',
 'B72PP2A_Ca_D32p34',
 'B72PP2A_Ca_D32p75',
 'B72PP2A_Ca_pARPP21',
 'B72PP2A_D32p34',
 'B72PP2A_D32p75',
 'B72PP2A_pARPP21',
 'CDK5',
 'CDK5_D32',
 'Ca',
 'CaM',
 'CaMKII',
 'CaMKII_CaM',
 'CaMKII_CaM_Ca2',
 'CaMKII_CaM_Ca2_psd',
 'CaMKII_CaM_Ca2_psd_Substrate',
 'CaMKII_CaM_Ca4',
 'CaMKII_CaM_Ca4_CaMKII_CaM_Ca4',
 'CaMKII_CaM_Ca4_psd',
 'CaMKII_CaM_Ca4_psd_CaMKII_CaM_Ca4_psd',
 'CaMKII_CaM_Ca4_psd_Substrate',
 'CaMKII_CaM_psd',
 'CaMKII_CaM_psd_Substrate',
 'CaMKII_psd',
 'CaM_Ca2',
 'CaM_Ca4',
 'CaM_Ca4_pARPP21',
 'D1R',
 'D1R_DA',
 'D1R_Golf',
 'D1R_Golf_DA',
 'D32',
 'D32p34',
 'D32p75',
 'DA',
 'GaolfGDP',
 'GaolfGTP',
 'Gbgolf',
 'Golf',
 'PDE10c',
 'PDE10c_cAMP',
 'PDE10r',
 'PDE10r_cAMP',
 'PDE4'

In [5]:
data_types = nd.list_data_types(0)
all_species_data = nd.get_all_data(neuron_id=0, exclude=["spikes", "voltage"])
time = nd.get_time()
voltage = nd.get_data("voltage", [0])

In [6]:
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "iframe"  # Do not save plots in the notebook, they can get BIG

fig = go.Figure()
for data_type in all_species_data:
    idx = time >= 0.0
    fig.add_trace(go.Scatter(x=time[idx], y=all_species_data[data_type][0][0].T[0][idx], name=data_type, line={"width":4}))

"""
fig.update_layout(xaxis_title="Time (s)", yaxis_title="Concentration", width=1000, height=800,
                 font={"size":18},  # General font size for all elements
                 legend={"font":{"size":16}},  # Specific font size for legend
                 xaxis={"title":{"font":{"size":20}}, "tickfont":{"size":14}},  # X-axis title and tick labels
                 yaxis={"title":{"font":{"size":20}}, "tickfont":{"size":14}})   # Y-axis title and tick labels
"""
fig.show()

In [7]:
# Reporting plot

import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "iframe"  # Do not save plots in the notebook, they can get BIG

fig = go.Figure()
for data_type in ["D1R_DA", "PKAc", "cAMP"]: #all_species_data:
    idx = time >= 0.0
    fig.add_trace(go.Scatter(x=time[idx], y=all_species_data[data_type][0][0].T[0][idx], name=data_type, line={"width":4}))

fig.update_layout(xaxis_title="Time (s)", yaxis_title="Concentration (mM)", width=1000, height=800,
                  font={"size":18},  # General font size for all elements
                  legend={"font":{"size":50}},  # Specific font size for legend
                  xaxis={"title":{"font":{"size":50}}, "tickfont":{"size":40}},  # X-axis title and tick labels
                  yaxis={"title":{"font":{"size":50}}, "tickfont":{"size":40}})   # Y-axis title and tick labels


fig.show()

fig.write_image("PKAc_estimate_4Hz-1.pdf", width=1200, height=800)
fig.write_image("PKAc_estimate_4Hz-1.png", width=1200, height=800)
# fig.write_image("bevan_1-example-da-cascade-1.pdf", width=1200, height=800)


fig2 = go.Figure()
for data_type in ["cal12_ms.modulation_factor", "cal13_ms.modulation_factor", 
                  "kas_ms.modulation_factor", "kir_ms.modulation_factor", "naf_ms.modulation_factor", ]: #all_species_data:
    idx = time >= 0.0
    data_type_str = data_type.replace("modulation_factor", "modulation")
    fig2.add_trace(go.Scatter(x=time[idx], y=all_species_data[data_type][0][0].T[0][idx], name=data_type_str, line={"width":4}))

fig2.update_layout(xaxis_title="Time (s)", yaxis_title="Modulation factor", width=1000, height=800,
                  font={"size":18},  # General font size for all elements
                  legend={"font":{"size":50}},  # Specific font size for legend
                  xaxis={"title":{"font":{"size":40}}, "tickfont":{"size":30}},  # X-axis title and tick labels
                  yaxis={"title":{"font":{"size":40}}, "tickfont":{"size":30}})   # Y-axis title and tick labels

fig2.show()
fig2.write_image("PKAc_estimate_4Hz-modulation.pdf", width=1200, height=800)
fig2.write_image("PKAc_estimate_4Hz-modulation..png", width=1200, height=800)
#fig2.write_image("bevan_fig1_biochem.pdf", width=1200, height=800)

nd = None

In [8]:
from snudda.utils import SnuddaLoadSimulation

nd_high = SnuddaLoadSimulation(sim_output_neuromodulation_high)
data_types = nd_high.list_data_types(0)
all_species_data = nd_high.get_all_data(neuron_id=0, exclude=["spikes", "voltage"])
time_high = nd_high.get_time()
voltage_high = nd_high.get_data("voltage")

pkac_high = nd_high.get_data("PKAc")

Loading networks/check_PKAc/simulation/output_neuromodulation_PKAc_20Hz.hdf5
WARNING. Depolarisation block in neuron - neuron_id: (name, parameter_key, morphology_key):
0: (dspn_0, pcfabf7dd, m4a48752c)
1: (dspn_1, pa925bef7, mfd0a1315)
2: (dspn_2, pb7178977, m93f282f3)
3: (dspn_3, p31fdf560, mfa814971)
4: (dspn_3, pd45a06eb, me61ede90)


In [9]:
nd_low = SnuddaLoadSimulation(sim_output_neuromodulation_low)
time_low = nd_low.get_time()
voltage_low = nd_low.get_data("voltage")
pkac_low = nd_low.get_data("PKAc")

Loading networks/check_PKAc/simulation/output_neuromodulation_PKAc_4Hz.hdf5
WARNING. Depolarisation block in neuron - neuron_id: (name, parameter_key, morphology_key):
0: (dspn_0, pcfabf7dd, m4a48752c)
1: (dspn_1, pa925bef7, mfd0a1315)
2: (dspn_2, pb7178977, m93f282f3)
3: (dspn_3, p31fdf560, mfa814971)
4: (dspn_3, pd45a06eb, me61ede90)


In [10]:
import numpy as np
import json

with open("../../data/JSON/estimate_PKAc_4Hz.json", "r") as f:
    config_low = json.load(f)

da_time_low = np.array(config_low["bath_application"]["DA"]["time"])
da_conc_low = np.array(config_low["bath_application"]["DA"]["concentration"]) * 1e6

with open("../../data/JSON/estimate_PKAc_20Hz.json", "r") as f:
    config_high = json.load(f)

da_time_high = np.array(config_high["bath_application"]["DA"]["time"])
da_conc_high = np.array(config_high["bath_application"]["DA"]["concentration"]) * 1e6

In [22]:
import plotly.graph_objects as go

# import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "iframe"  # Do not save plots in the notebook, they can get BIG


for neuron_id in [0, 1]:
    
    fig = go.Figure()
    
    sct_low = go.Scatter(x=time_low, y=pkac_low[0][neuron_id][:,0]*1e6, name="DA (4Hz)", opacity=0.5,  line={"width":4, "color":"black"}, yaxis="y1")
    sct_high = go.Scatter(x=time_high, y=pkac_high[0][neuron_id][:,0]*1e6, name="DA (20Hz)", opacity=0.5,  line={"width":4, "color":"red"}, yaxis="y1")
    
    fig.add_traces([sct_low, sct_high])

    fig.update_layout(
        xaxis_title="Time (s)",
        yaxis_title="PKAc (nM)",
        width=800,
        height=600,
        font=dict(size=30),
        legend=dict(font=dict(size=30)),
    
        plot_bgcolor="white",
        paper_bgcolor="white",
    
        xaxis=dict(
            title=dict(font=dict(size=36)),
            tickfont=dict(size=30),
            range=[0, 8],
            dtick=2,
            showline=True,
            linecolor="black",
            linewidth=3,
            showgrid=False,
            zeroline=False,
            ticks="outside",
            tickwidth=2,
            tickcolor="black"
        ),
    
        yaxis=dict(
            title=dict(
                text="PKAc (nM)",
                font=dict(size=36)
            ),
            tickfont=dict(size=30),
            side="left",
            showline=True,
            linecolor="black",
            linewidth=3,
            showgrid=False,
            zeroline=False,
            ticks="outside",
            tickwidth=2,
            tickcolor="black"
        ),
    )
    fig.show()
    fig.write_image(f"Estimate-PKAc-figure-{neuron_id}.pdf", scale=1, height=600, width=800)
    fig.write_image(f"Estimate-PKAc-figure-{neuron_id}.png", scale=1, height=600, width=800)
